In [40]:

import pandas as pd
import numpy as np
from pandas import read_csv
from pmdarima.arima import auto_arima
from datetime import datetime
import matplotlib.pyplot as plt
from pmdarima.arima import ADFTest

In [41]:
values = read_csv("../../Time_series_data.csv")

In [42]:
# values = read_csv("../../../daily-minimum-temperatures-in-me.csv")

In [43]:
# values = read_csv("../../Electric_Production.csv")

In [44]:
print(values.shape[0])

105


In [45]:
values = values.rename(columns={values.columns[1]: 'Data'})
values = values.rename(columns={values.columns[0]: 'Time'})

In [46]:
x = values['Time'][values.shape[0]-1]
print(x)

2021-09


In [47]:
values['Time'] = pd.to_datetime(values['Time'], errors = 'coerce')
values.set_index('Time', inplace=True)

In [48]:
if isinstance(values['Data'][0], str):
    print('DUy')

In [49]:
print(values)

            Data
Time            
2013-01-01  2815
2013-02-01  2672
2013-03-01  2755
2013-04-01  2721
2013-05-01  2946
...          ...
2021-05-01  4618
2021-06-01  5312
2021-07-01  4298
2021-08-01  1413
2021-09-01  5877

[105 rows x 1 columns]


In [50]:
# values['Data'] = values['Data'].apply(lambda x: float(x) if x.replace('.', '', 1).isdigit() else None)

In [51]:
missing_values_count = values.isna().sum()
print(missing_values_count)

Data    0
dtype: int64


In [52]:
# Tạo một Series Boolean cho các dòng có giá trị NaN
nan_rows = values['Data'].isna()

# Truy xuất chỉ mục của các dòng có giá trị NaN
rows_with_nan = values[nan_rows].index
print(rows_with_nan)

DatetimeIndex([], dtype='datetime64[ns]', name='Time', freq=None)


In [53]:
values['Data'].fillna(values['Data'].mean(), inplace=True)

In [54]:
values

,Data
Time,
2013-01-01,2815
2013-02-01,2672
2013-03-01,2755
2013-04-01,2721
2013-05-01,2946
...,...
2021-05-01,4618
2021-06-01,5312
2021-07-01,4298


In [55]:
missing_values_count = values.isna().sum().sum()
if missing_values_count != 0:
    values = values[~(values.isna().any(axis=1))]
    print(values)

In [56]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(values, test_size=0.2, shuffle=False)

arima_model = auto_arima(train, start_p=0, d=1, start_q=0,
                        max_p=5, max_d=5, max_q=5, start_P=0,
                        D=1, start_Q=0, max_P=5, max_D=5,
                        max_Q=5, m=12, seasonal=True,
                        error_action='warn', trace=True,
                        supress_warnings=True, stepwise=True,
                        random_state=20, n_fits=50)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=1183.693, Time=0.04 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=1173.736, Time=0.21 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=1157.042, Time=0.57 sec


 ARIMA(0,1,1)(0,1,0)[12]             : AIC=1155.109, Time=0.21 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=1157.009, Time=0.48 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=1158.348, Time=1.22 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=1155.379, Time=0.33 sec
 ARIMA(0,1,2)(0,1,0)[12]             : AIC=1155.138, Time=0.27 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=1173.612, Time=0.06 sec
 ARIMA(1,1,2)(0,1,0)[12]             : AIC=1155.790, Time=0.44 sec
 ARIMA(0,1,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.21 sec

Best model:  ARIMA(0,1,1)(0,1,0)[12]          
Total fit time: 4.042 seconds


In [57]:
test.index

DatetimeIndex(['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
               '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
               '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
               '2021-09-01'],
              dtype='datetime64[ns]', name='Time', freq=None)

In [58]:
prediction = pd.DataFrame(
    arima_model.predict(n_periods=30),index=test.index )
prediction.columns = ['predicted_values']
prediction.reset_index(inplace=True)

ValueError: Shape of passed values is (30, 1), indices imply (21, 1)

In [ ]:
print(prediction)

In [ ]:
index_future_dates = pd.date_range(
    start=x, end='2023-01-1', freq='MS')

In [ ]:
prediction_arima = pd.DataFrame(
    arima_model.predict(n_periods=26), index = index_future_dates)
prediction_arima.columns = ['predicted_values']  
print(prediction_arima)